<a href="https://colab.research.google.com/github/ankit-genzeon/AI-ML-Bootcamp-Genzeon-2023/blob/master/Bot_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
Dataset = '/content/drive/MyDrive/Colab Notebooks/DataSet/bot_detection_data.csv'
df = pd.read_csv(Dataset)

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
df.shape

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Keep the desired columns
df = df[['Tweet', 'Username', 'Bot Label', 'Verified', 'Hashtags', 'Follower Count', 'Retweet Count']]

# Visualize the relationship between Retweet Count and Follower Count
sns.scatterplot(data=df, x='Retweet Count', y='Follower Count')
plt.title('Relationship between Retweet Count and Follower Count')
plt.show()

# Count of Verified and Unverified Accounts
sns.countplot(data=df, x='Verified')
plt.title('Count of Verified and Unverified Accounts')
plt.xlabel('Verified')
plt.ylabel('Count')
plt.show()


In [5]:
from sklearn.preprocessing import LabelEncoder
# Label encode the Verified column
label_encoder = LabelEncoder()
df['Verified'] = label_encoder.fit_transform(df['Verified'])

In [7]:
from sklearn.preprocessing import LabelEncoder
# Handling missing values
df['Hashtags'].fillna('Unknown', inplace=True)  # Replace missing locations with 'Unknown'

In [ ]:
df.head()

In [9]:
from scipy.sparse import csr_matrix
from sklearn.feature_extraction.text import TfidfVectorizer

# Combine all text columns into a single Series
text_data = df['Tweet'] + ' ' + df['Username'] + ' ' + df['Hashtags']

# Text vectorization using TF-IDF (sparse representation)
vectorizer = TfidfVectorizer()
text_sparse = vectorizer.fit_transform(text_data)

# # Split the sparse matrix into separate parts
# tweet_sparse = text_sparse[:, :tweet_length]
# username_sparse = text_sparse[:, tweet_length:(tweet_length+username_length)]
# hashtags_sparse = text_sparse[:, (tweet_length+username_length):]

# # Print the lengths
# print("Tweet Length:", tweet_length)
# print("Username Length:", username_length)
# print("Hashtags Length:", hashtags_length)


In [12]:
# Keep the desired columns
df = df[['Username','Tweet', 'Verified', 'Hashtags', 'Follower Count', 'Retweet Count', 'Bot Label']]

In [13]:
df.head()

,Username,Tweet,Verified,Hashtags,Follower Count,Retweet Count,Bot Label
0,flong,Station activity person against natural majori...,0,Unknown,2353,85,1
1,hinesstephanie,Authority research natural life material staff...,1,both live,9617,55,0
2,roberttran,Manage whose quickly especially foot none to g...,1,phone ahead,4363,6,0
3,pmason,Just cover eight opportunity strong policy which.,1,ever quickly new I,2242,54,1
4,noah87,Animal sign six data good or.,0,foreign mention,8438,26,1
